<a href="https://colab.research.google.com/github/Youssef-Hossam5/Hand-Gesture-Recogonition/blob/main/Hand_Gestures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the required python libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import xgboost as xgb
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Dataset Loading


In [ ]:
data_path='/content/drive/MyDrive/Hand Gesture Classification/hand_landmarks_data.csv' #drive must be mounted before running
raw_data=pd.read_csv(data_path)
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25675 entries, 0 to 25674
Data columns (total 64 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      25675 non-null  float64
 1   y1      25675 non-null  float64
 2   z1      25675 non-null  float64
 3   x2      25675 non-null  float64
 4   y2      25675 non-null  float64
 5   z2      25675 non-null  float64
 6   x3      25675 non-null  float64
 7   y3      25675 non-null  float64
 8   z3      25675 non-null  float64
 9   x4      25675 non-null  float64
 10  y4      25675 non-null  float64
 11  z4      25675 non-null  float64
 12  x5      25675 non-null  float64
 13  y5      25675 non-null  float64
 14  z5      25675 non-null  float64
 15  x6      25675 non-null  float64
 16  y6      25675 non-null  float64
 17  z6      25675 non-null  float64
 18  x7      25675 non-null  float64
 19  y7      25675 non-null  float64
 20  z7      25675 non-null  float64
 21  x8      25675 non-null  float64
 22

# Data Visualization

In [ ]:
# Skeleton connections
connections = [
    (1,2),(2,3),(3,4),(4,5),
    (1,6),(6,7),(7,8),(8,9),
    (1,10),(10,11),(11,12),(12,13),
    (1,14),(14,15),(15,16),(16,17),
    (1,18),(18,19),(19,20),(20,21),
    (6,10),(10,14),(14,18)
]

# Function for prepare coordinates (flip axes if needed)
def prepare_coords(sample, flip_x=True, flip_y=True):
    xs = [sample[f'x{i}'] for i in range(1, 22)]
    ys = [sample[f'y{i}'] for i in range(1, 22)]
    if flip_x:
        xs = [max(xs) - x for x in xs]
    if flip_y:
        ys = [max(ys) - y for y in ys]
    return xs, ys

# data sample for each class
for cls in sorted(raw_data['label'].unique()):
    sample = raw_data[raw_data['label'] == cls].iloc[0]  # pick first sample of class
    xs, ys = prepare_coords(sample)

    fig = go.Figure()

    # Adding points
    fig.add_trace(go.Scatter(
        x=xs, y=ys,
        mode='markers',
        marker=dict(size=8, color='red'),
        name=f'Class {cls}'
    ))

    # Adding connections
    for start, end in connections:
        fig.add_trace(go.Scatter(
            x=[xs[start-1], xs[end-1]],
            y=[ys[start-1], ys[end-1]],
            mode='lines',
            line=dict(color='green', width=3),
            showlegend=False
        ))

    fig.update_layout(
        title=f"2D Hand Skeleton - Class {cls}",
        xaxis=dict(scaleanchor="y", scaleratio=1),
        yaxis=dict(scaleanchor="x", scaleratio=1),
        width=600,
        height=600
    )

    fig.show()


#  Data Preprocessing

In [ ]:
def normalize_hand_row(row):
    # Wrist = landmark 1
    wrist_x = row['x1']
    wrist_y = row['y1']

    # Middle finger tip = landmark 12
    mid_x = row['x12'] - wrist_x
    mid_y = row['y12'] - wrist_y

    scale = np.sqrt(mid_x**2 + mid_y**2)
    if scale == 0:
        scale = 1e-6  # avoid division by zero

    new_row = {}

    for i in range(1, 22):
        # recenter
        x = row[f'x{i}'] - wrist_x
        y = row[f'y{i}'] - wrist_y

        # scale
        new_row[f'x{i}'] = x / scale
        new_row[f'y{i}'] = y / scale

        # z unchanged
        new_row[f'z{i}'] = row[f'z{i}']

    return pd.Series(new_row)
X_norm = raw_data.apply(normalize_hand_row, axis=1)
le = LabelEncoder()
X = X_norm.values
y = le.fit_transform(raw_data['label'].values)
print("X shape:", X.shape)  # (n_samples=25675, 21*3)
print("y shape:", y.shape)  # (n_samples,)

X shape: (25675, 63)
y shape: (25675,)


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,)
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Accuracy: 0.9801363193768257
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99       301
           1       1.00      1.00      1.00       259
           2       0.99      0.99      0.99       189
           3       0.98      0.99      0.98       327
           4       0.98      0.99      0.99       287
           5       0.96      0.96      0.96       217
           6       0.99      0.99      0.99       318
           7       0.95      0.97      0.96       253
           8       0.98      0.98      0.98       330
           9       0.98      0.97      0.97       288
          10       0.99      0.97      0.98       299
          11       1.00      0.99      0.99       292
          12       0.94      0.97      0.96       296
          13       0.98      0.98      0.98       314
          14       0.99      0.96      0.98       291
          15       0.99      0.98      0.99       331
          16       0.98     

In [ ]:

svm_model = SVC(kernel='rbf', C=10, gamma='scale', random_state=42)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Classification Report:\n", classification_report(y_test, y_pred_svm))

Accuracy: 0.9848101265822785
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       301
           1       1.00      1.00      1.00       259
           2       0.99      0.99      0.99       189
           3       0.99      0.99      0.99       327
           4       0.99      0.99      0.99       287
           5       0.96      0.97      0.97       217
           6       1.00      0.99      0.99       318
           7       0.96      0.97      0.96       253
           8       0.98      0.99      0.99       330
           9       0.99      0.96      0.97       288
          10       1.00      0.98      0.99       299
          11       1.00      0.99      0.99       292
          12       0.95      0.98      0.97       296
          13       0.99      0.98      0.98       314
          14       0.99      0.97      0.98       291
          15       0.99      0.99      0.99       331
          16       0.96     

In [ ]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("Classification Report:\n", classification_report(y_test, y_pred_xgb))

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning:

[18:38:12] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.




Accuracy: 0.9824732229795521
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       301
           1       1.00      1.00      1.00       259
           2       0.99      0.99      0.99       189
           3       0.98      0.98      0.98       327
           4       0.99      0.99      0.99       287
           5       0.96      0.97      0.96       217
           6       0.99      0.99      0.99       318
           7       0.96      0.97      0.96       253
           8       0.98      0.98      0.98       330
           9       0.98      0.97      0.98       288
          10       0.98      0.98      0.98       299
          11       1.00      0.99      0.99       292
          12       0.95      0.97      0.96       296
          13       0.98      0.99      0.99       314
          14       0.99      0.97      0.98       291
          15       0.99      0.98      0.99       331
          16       0.97     

all three models are nearly similar in accuracy and macro avg and weighted avg but svm is slightly higher and thats expected as it has ability to smooth non linear decision boundaries so i will proceed with svm model with RBF kernel

In [ ]:
joblib.dump(svm_model, "/content/drive/MyDrive/Hand Gesture Classification/svm_hand_gesture_model.pkl")
joblib.dump(le, '/content/drive/MyDrive/Hand Gesture Classification/label_encoder.pkl')
print(f"Classes: {le.classes_}")
print(f"Number of classes: {len(le.classes_)}")

Classes: ['call' 'dislike' 'fist' 'four' 'like' 'mute' 'ok' 'one' 'palm' 'peace'
 'peace_inverted' 'rock' 'stop' 'stop_inverted' 'three' 'three2' 'two_up'
 'two_up_inverted']
Number of classes: 18
